# KERAS-BERT: predict masked words
This is the KERAS-BERT python module that is accessible through R. 
Code modified from this demo:
https://github.com/CyberZHG/keras-bert/blob/master/demo/load_model/keras_bert_load_and_predict.ipynb

Download Pretrained Weights

In [1]:
!pip install -q keras-bert

In [2]:
!wget -q https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip

/bin/bash: wget: command not found


In [3]:
!unzip -o chinese_L-12_H-768_A-12.zip

unzip:  cannot find or open chinese_L-12_H-768_A-12.zip, chinese_L-12_H-768_A-12.zip.zip or chinese_L-12_H-768_A-12.zip.ZIP.


### 1. Build Model & Dictionary

Set paths:

In [4]:
import os
pretrained_path = '../../../../uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

Enable tf.keras by adding TF_KERAS to environment variables:

In [5]:
os.environ['TF_KERAS'] = '1'

Build the dictionary & inverse dictionary:

In [6]:
import codecs

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
token_dict_inv = {v: k for k, v in token_dict.items()}

Build the model:

In [7]:
from keras_bert import load_trained_model_from_checkpoint
model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True)
model.summary(line_length=120)

/Users/kelichiu/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kelichiu/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kelichiu/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/kelichiu/opt/anaconda3/lib/python3.7/site-packages/ten

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/Users/kelichiu/opt/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kelichiu/opt/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kelichiu/opt/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/kelichiu/opt/anaconda3/lib/python

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
Input-Token (InputLayer)               [(None, 512)]              0                                                     
________________________________________________________________________________________________________________________
Input-Segment (InputLayer)             [(None, 512)]              0                                                     
________________________________________________________________________________________________________________________
Embedding-Token (TokenEmbedding)       [(None, 512, 768), (30522, 23440896      Input-Token[0][0]                       
________________________________________________________________________________________________________________________
Embedding-Segment

### 2. Masked certain words
Masked the word "intention" (token 13) and "Kingdom" (token 33) to be predicted. Here, the masked words are not errors and are selected manually. In the text cleaning process, the words should be picked by a spell checker or other error detection proccess.

In [19]:
from keras_bert import Tokenizer
tokenizer = Tokenizer(token_dict)
text = 'His Majesty’s Government wish to add that they have no intention of requesting the establishment of military bases in peace time within the area of Palestine now united to the Kingdom of Jordan .'
tokens = tokenizer.tokenize(text)
#tokens[13] = tokens[33] = '[MASK]'
print('Tokens:', tokens)

Tokens: ['[CLS]', 'his', 'majesty', '’', 's', 'government', 'wish', 'to', 'add', 'that', 'they', 'have', 'no', 'int', '##ion', 'of', 'requesting', 'the', 'establishment', 'of', 'military', 'bases', 'in', 'peace', 'time', 'within', 'the', 'area', 'of', 'palestine', 'now', 'united', 'to', 'the', 'dom', 'of', 'jordan', '.', '[SEP]']


Build arrays of indices tokens(word index), segments tokens(sentence segments), and mask indices as the model inputs. Due to the limitation of the package, the arrays are made manually. The three arrays have the same shape, which is 512 by 512 (set by the tutorial, as the maximum input length). Each element of the array represent a token. We do not have 512 tokens here so there are unused space in the array. 

In [9]:
import numpy as np

indices = np.array([[token_dict[token] for token in tokens] + [0] * (512 - len(tokens))])
segments = np.array([[0] * len(tokens) + [0] * (512 - len(tokens))])
masks = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]])

Set the prediction matrix and predict the masked words. The prediction matrix will give us a index of the predicted word, the index is then used to look for the corresponding word through the inverse token dictionary. Here, the predictions of the two masked words are "intention" and "state". The original words are "intention" and "Kingdom".

In [24]:
predicts = model.predict([indices, segments, masks])
predicts_dict = predicts[0].argmax(axis=-1).tolist()
print('Fill with: ', token_dict_inv[predicts_dict[0][13]], token_dict_inv[predicts_dict[0][33]])

Fill with:  intention state
